In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [108]:
data = pd.read_csv('data_merged.csv')

In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3381226 entries, 0 to 3381225
Data columns (total 22 columns):
Unnamed: 0             int64
trip_id                int64
start_time             object
end_time               object
bikeid                 int64
tripduration           int64
from_station_id        int64
from_station_name      object
to_station_id          int64
to_station_name        object
usertype               object
gender                 object
month                  int64
date                   object
hour                   int64
day                    int64
dayofweek              int64
weekend                int64
age                    float64
DailySnowDepth         float64
HourlyTemperature      float64
HourlyPrecipitation    float64
dtypes: float64(4), int64(11), object(7)
memory usage: 567.5+ MB


In [110]:
data.drop(columns='Unnamed: 0',inplace=True)

In [111]:
checkout_count = pd.DataFrame(data.groupby(['from_station_id','date','month','day', 'hour'])['trip_id'].count())

In [112]:
checkout_count.reset_index(inplace=True)

In [113]:
checkout_count.rename(columns={'trip_id':'checkouts'})

,from_station_id,date,month,day,hour,checkouts
0,2,2018-01-06,1,6,13,1
1,2,2018-01-09,1,9,18,1
2,2,2018-01-11,1,11,11,4
3,2,2018-01-11,1,11,16,1
4,2,2018-01-13,1,13,15,2
5,2,2018-01-14,1,14,12,1
6,2,2018-01-20,1,20,14,9
7,2,2018-01-20,1,20,15,4
8,2,2018-01-20,1,20,16,1
9,2,2018-01-20,1,20,17,5


In [114]:
checkout_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1191060 entries, 0 to 1191059
Data columns (total 6 columns):
from_station_id    1191060 non-null int64
date               1191060 non-null object
month              1191060 non-null int64
day                1191060 non-null int64
hour               1191060 non-null int64
trip_id            1191060 non-null int64
dtypes: int64(5), object(1)
memory usage: 54.5+ MB


In [39]:
data_checkout.shape

(1191060, 5)

In [115]:
#importing weather data for 2018 chicago
weather = pd.read_csv('weather1.csv')

#Snow details are not available on hourly level only day level saving to new df
snow_day = weather[weather.REPORT_TYPE == 'SOD  ']


#cleaning weather data 
weather = weather[weather.REPORT_TYPE != 'SOD  '] #snow day
weather = weather[weather.REPORT_TYPE != 'SOM  '] #snow month 
weather = weather[weather.REPORT_TYPE != 'SY-MT'] 
weather = weather[weather.REPORT_TYPE != 'FM-12'] 
weather = weather[weather.REPORT_TYPE != 'FM-16'] 


#Keeping only relevant columns in weather
weather = weather[['DATE','HourlyDryBulbTemperature','HourlyPrecipitation']]
weather.rename(columns={'HourlyDryBulbTemperature':'HourlyTemperature'},inplace=True)

#Extracting  month ,day ,hour from DATE column
weather['month'] = pd.to_datetime(weather.DATE,format='%Y-%m-%d').dt.month
weather['day'] = pd.to_datetime(weather.DATE,format='%Y-%m-%d').dt.day
weather['hour']=pd.to_datetime(weather.DATE,format='%Y-%m-%d').dt.hour
weather.drop(columns='DATE',inplace=True)



#converting HourlyPrecipitation with value 'T'traces to 0.01 and nan with ffill
weather[weather['HourlyPrecipitation'] == 'T'] = 0.001
weather['HourlyPrecipitation'] = weather['HourlyPrecipitation'].astype(float)

weather['HourlyPrecipitation']=weather['HourlyPrecipitation'].fillna(method='ffill')

weather['HourlyTemperature'] = weather['HourlyTemperature'].fillna(method='ffill')

#Cleaned weather data 2018
weather.to_csv('weather_final.csv')

#selecting relevent columns from snow_day data frame
snow_day = snow_day[['DATE','DailySnowDepth']]

#Extracting date from DATE 
#snow_day['date'] = pd.to_datetime(snow_day.DATE,format='%Y-%m-%d').dt.date
snow_day['month'] = pd.to_datetime(snow_day.DATE,format='%Y-%m-%d').dt.month
snow_day['day'] = pd.to_datetime(snow_day.DATE,format='%Y-%m-%d').dt.day
snow_day.drop(columns='DATE',inplace=True)
snow_day.to_csv('snow.csv')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (41,49,51,75,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [116]:
data_checkout = checkout_count.merge(snow_day ,on=['month','day'])

In [117]:
data_checkout = data_checkout.merge(weather,on=['month','day','hour'])

D:\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:946: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


In [118]:
data_checkout.index = data_checkout['date']

In [119]:
data_checkout.head(10)

,from_station_id,date,month,day,hour,trip_id,DailySnowDepth,HourlyTemperature,HourlyPrecipitation
date,,,,,,,,,
2018-01-06,2,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,3,2018-01-06,1,6,13,3,1.0,14.0,0.0
2018-01-06,14,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,23,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,24,2018-01-06,1,6,13,3,1.0,14.0,0.0
2018-01-06,26,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,28,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,35,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,38,2018-01-06,1,6,13,5,1.0,14.0,0.0


In [120]:
data_checkout.rename(columns={'trip_id':'checkouts'},inplace=True)

In [121]:
data_checkout.head()

,from_station_id,date,month,day,hour,checkouts,DailySnowDepth,HourlyTemperature,HourlyPrecipitation
date,,,,,,,,,
2018-01-06,2,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,3,2018-01-06,1,6,13,3,1.0,14.0,0.0
2018-01-06,14,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,23,2018-01-06,1,6,13,1,1.0,14.0,0.0
2018-01-06,24,2018-01-06,1,6,13,3,1.0,14.0,0.0


In [122]:
data_checkout.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1191060 entries, 2018-01-06 to 2018-01-12
Data columns (total 9 columns):
from_station_id        1191060 non-null int64
date                   1191060 non-null object
month                  1191060 non-null int64
day                    1191060 non-null int64
hour                   1191060 non-null int64
checkouts              1191060 non-null int64
DailySnowDepth         1191060 non-null float64
HourlyTemperature      1191060 non-null float64
HourlyPrecipitation    1191060 non-null float64
dtypes: float64(3), int64(5), object(1)
memory usage: 90.9+ MB


In [123]:
y = data_checkout['checkouts']
data_checkout.drop(columns=['checkouts','date'],inplace=True)

In [124]:
xtrain,xtest,ytrain,ytest = train_test_split(data_checkout,y,test_size=0.3)

In [142]:
model = LinearRegression()

In [143]:
model.fit(xtrain,ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [144]:
y_pred = model.predict(xtest)

In [145]:
y_pred

array([2.78263894, 3.17913107, 2.27415123, ..., 3.5165859 , 2.54811718,
       4.40011596])

In [149]:
# conf_matrix = metrics.confusion_matrix(ytest,y_pred)
# sns.heatmap(conf_matrix, annot = True, fmt = ".3f", square = True, cmap = plt.cm.Blues)
# plt.ylabel('Actual')
# plt.xlabel('Predicted')
# plt.title('Confusion matrix')
# plt.tight_layout()

accuracy = metrics.accuracy_score(ytest, y_pred)
error = 1 - accuracy
precision = metrics.precision_score(ytest, y_pred, average = None)
recall = metrics.recall_score(ytest, y_pred, average = None)
F1_score = metrics.f1_score(ytest, y_pred, average = None)
print([accuracy, error, precision, recall, F1_score])

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [0]:
## Check the metrics of the Linear Regression model


# Mean Squared Error
lm_mse = mean_squared_error(ytest,y_pred)

# r2 score
lm_r2 = r2_score(ytest,y_pred)

# Mean Absolute Error
lm_mean_ae = mean_absolute_error(ytest,y_pred)

# Median Absolute Error
lm_median_ae = median_absolute_error(ytest,y_pred)

a = metrics.accuracy_score(ytest,y_pred)
print(a)

In [136]:

print('Mean Squared Error =', lm_mse)
print('r2 =', lm_r2)
print('Mean Absolute Error=', lm_mean_ae)
print('Median Absolute Error', lm_median_ae)


Mean Squared Error = 16.48542954658469
r2 = 0.05628954977154854
Mean Absolute Error= 1.9780799544672694
Median Absolute Error 1.3789056507020374


In [137]:
lasso = Lasso(normalize=True)
lasso.fit(xtrain, ytrain)
y_pred = lasso.predict(xtest)

In [138]:


# Mean Squared Error
lm_mse = mean_squared_error(ytest,y_pred)

# r2 score
lm_r2 = r2_score(ytest,y_pred)

# Mean Absolute Error
lm_mean_ae = mean_absolute_error(ytest,y_pred)

# Median Absolute Error
lm_median_ae = median_absolute_error(ytest,y_pred)

In [141]:

print('Mean Squared Error =', lm_mse)
print('r2 =', round(lm_r2,5))
print('Mean Absolute Error=', lm_mean_ae)
print('Median Absolute Error', lm_median_ae)

Mean Squared Error = 17.46885480886229
r2 = -1e-05
Mean Absolute Error= 2.088292617435617
Median Absolute Error 1.8355798316505587
